In [2]:
import numpy as np
from tqdm import trange

In [ ]:
# load data from video
import cv2

# resolution
W = 1280
H = 960

def get_video_data(video_path, log_path):
    frames = []
    
    with open(log_path, "r") as log_file:
        labels = log_file.read().split("\n")
        log_file.close()

    cap = cv2.VideoCapture(video_path)
    idx = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (W,H))
            frames.append(frame)
            cv2.imshow('frame %d'%idx, frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
            idx += 1
        else:
            break
    cap.release()
    cv2.destroyAllWindows()
    
    return np.array(frames), np.array(labels)


data_path = "data/videos/with_crossroads/"  # CHANGE THIS
video_name = "city_1.mp4"  # CHANGE THIS
log_path = data_path + video_name[:-4] + ".txt"

frames, labels = get_video_data(data_path+video_name, log_path)

In [5]:
frames.shape

(71, 540, 960, 3)

In [6]:
# create model
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # image size
        self.W = 1920//2
        self.H = 1080//2

        # Convolutional layers
        # TODO: decide layers architecture
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # Convolutional layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)   # output is binary (is or is not crossroad)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [7]:
# train
model = Net()